In [5]:
!pip install beautifulsoup4==4.13.4
!pip install branca==0.8.1
!pip install certifi==2025.4.26
!pip install charset-normalizer==3.4.1
!pip install folium==0.19.5
!pip install geopandas==1.0.1
!pip install idna==3.10
!pip install Jinja2==3.1.6
!pip install lxml==5.4.0
!pip install MarkupSafe==3.0.2
!pip install numpy==2.2.5
!pip install packaging==25.0
!pip install pandas==2.2.3
!pip install pyogrio==0.10.0
!pip install pyproj==3.7.1
!pip install python-dateutil==2.9.0.post0
!pip install pytz==2025.2
!pip install requests==2.32.3
!pip install shapely==2.1.0
!pip install six==1.17.0
!pip install soupsieve==2.7
!pip install typing_extensions==4.13.2
!pip install tzdata==2025.2
!pip install urllib3==2.4.0
!pip install xyzservices==2025.4.0

In [6]:
from datetime import datetime
import folium
import requests
from bs4 import BeautifulSoup as bs
from folium import plugins

In [7]:
koordinatList = []
buyuklukList = []
tarihList = []
saatList = []
derinlikList = []

In [8]:
r = requests.get('http://www.koeri.boun.edu.tr/scripts/lst0.asp')
soup = bs(r.content, 'html.parser')
pre_tag = soup.find("pre")
text = pre_tag.get_text()

In [9]:
lines = text.splitlines()
for line in lines:
    if line.strip() and line[0].isdigit():  # Tarih ile başlayan satırları filtrele
        tarih = line[0:10].strip()
        saat = line[11:19].strip()
        enlem = line[20:28].strip()
        boylam = line[29:37].strip()
        derinlik = line[38:50].strip()
        ml = line[60:64].strip()
        yer = line[67:113].strip()
        cozum = line[113:].strip()

        koordinatList.append((enlem, boylam))
        buyuklukList.append(ml)
        tarihList.append(tarih)
        saatList.append(saat)
        derinlikList.append(derinlik)

In [10]:
color_dict = {
    0: '#ffffff',
    1: '#ffffff',
    2: '#bfccff',
    3: '#99f',
    4: '#8ff',
    5: '#7df894',
    6: '#ff0',
    7: '#fd0',
    8: '#ff9100',
    9: '#f00'
}

In [12]:
m = folium.Map(location=[38.76534530779815, 35.44033727686726], zoom_start=6, min_zoom=6, tiles='OpenStreetMap')
minimap = plugins.MiniMap().add_to(m)
for i in range(len(koordinatList)):
    x_koordinati = float(koordinatList[i][0])
    y_koordinati = float(koordinatList[i][1])
    bilgi = '''
    Tarih: {}
    Saat: {}
    Derinlik: {} km
    Büyüklük: {} ML
    '''.format(tarihList[i], saatList[i], derinlikList[i], buyuklukList[i])

    iframe = folium.IFrame(bilgi)
    popup = folium.Popup(iframe, min_width=150, max_width=250)

    folium.Circle(radius=float(buyuklukList[i]) * 1000,
                  location=[x_koordinati, y_koordinati],
                  color='crimson',
                  fill_color=color_dict[int(float(buyuklukList[i]))],
                  popup=popup).add_to(m)

now = datetime.now()
# kayıt işlemi
m.save(f"son_{now.strftime("%Y-%m_%d_%H_%M_%S")}_deprem.html")